In [5]:
from ultralytics import YOLO

# YOLOv8n = nano version, paling cepat untuk CPU
model = YOLO("yolov8n.pt")

print("YOLOv8n pretrained model loaded!")

YOLOv8n pretrained model loaded!


In [6]:
import os

VIDEO_DIR = "../../../data"  

exts = [".mp4", ".avi", ".mov", ".mkv", ".webm"]

video_files = [
    f for f in os.listdir(VIDEO_DIR)
    if os.path.splitext(f)[1].lower() in exts
]

print("Video ditemukan:", len(video_files))
video_files


Video ditemukan: 7


['interview_question_1.webm',
 'interview_question_2.webm',
 'interview_question_3.webm',
 'interview_question_4.webm',
 'interview_question_5.webm',
 'wawancara1.mp4',
 'wawancara2.mp4']

In [7]:
import matplotlib.pyplot as plt

def show_frame(frame, title=""):
    plt.figure(figsize=(8, 8))
    plt.title(title)
    plt.imshow(frame[:, :, ::-1])  # BGR → RGB
    plt.axis("off")
    plt.show()


In [8]:
import cv2
import pandas as pd

results_summary = []

vid_name = "interview_question_1.webm"
video_path = os.path.join(VIDEO_DIR, vid_name)

print(f"\nProcessing video: {vid_name}")

cap = cv2.VideoCapture(video_path)

frame_id = 0
frame_rate = cap.get(cv2.CAP_PROP_FPS)  # fps video
print("FPS:", frame_rate)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1

    # YOLO inference
    results = model(frame)[0]

    # Count persons
    person_count = sum(1 for box in results.boxes if int(box.cls[0]) == 0)

    # Hitung timestamp (dalam detik)
    timestamp_sec = frame_id / frame_rate

    results_summary.append({
        "video": vid_name,
        "frame": frame_id,
        "timestamp_sec": timestamp_sec,
        "person_count": person_count,
    })

cap.release()
print("\nSelesai memproses video!")

# --- Convert ke dataframe
df = pd.DataFrame(results_summary)

# --- Filter orang > 1
df_multi_person = df[df["person_count"] > 1]

df_multi_person


Processing video: interview_question_1.webm
FPS: 16.583333333333332

0: 384x640 1 person, 1 chair, 80.7ms
Speed: 22.2ms preprocess, 80.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 56.9ms
Speed: 1.8ms preprocess, 56.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 54.2ms
Speed: 1.1ms preprocess, 54.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 54.2ms
Speed: 1.1ms preprocess, 54.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 54.3ms
Speed: 1.1ms preprocess, 54.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 56.1ms
Speed: 1.1ms preprocess, 56.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 chair, 56.4ms
Speed: 1.1ms preprocess, 56.4ms inference, 1.2ms postprocess per image at shape (1

,video,frame,timestamp_sec,person_count
291,interview_question_1.webm,292,17.60804,2


In [13]:
def format_timestamp(seconds):
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m:02d}:{s:02d}"

df_multi_person["timestamp_formatted"] = df_multi_person["timestamp_sec"].apply(format_timestamp)

df_multi_person.head()

,video,frame,timestamp_sec,person_count,timestamp_formatted
291,interview_question_1.webm,292,17.60804,2,00:17


In [10]:
df = pd.DataFrame(results_summary)
df.head()

,video,frame,timestamp_sec,person_count
0,interview_question_1.webm,1,0.060302,1
1,interview_question_1.webm,2,0.120603,1
2,interview_question_1.webm,3,0.180905,1
3,interview_question_1.webm,4,0.241206,1
4,interview_question_1.webm,5,0.301508,1


In [11]:
df.to_csv("people_detection_video_results.csv", index=False)
print("Saved → people_detection_video_results.csv")

Saved → people_detection_video_results.csv


In [12]:
df_summary = df.groupby("video")["person_count"].agg(
    total_frames="count",
    max_person="max",
    avg_person="mean"
)

df_summary

,total_frames,max_person,avg_person
video,,,
interview_question_1.webm,1544,2,1.000648
